# PDF파일 읽고 따라해보기

## 3. 데이터 가공
### 1. 필수 모듈 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium


### 2. 데이터 읽어들이기


In [2]:
bikes_temp = {}
for i in range(1,7):
    bikes_temp[i] = pd.read_csv(f"data/bike_rent_{i}.csv", encoding='utf-8',parse_dates=['대여일시'])
bikes = pd.concat(bikes_temp.values(), ignore_index=True)

In [3]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215632 entries, 0 to 2215631
Data columns (total 6 columns):
 #   Column    Dtype         
---  ------    -----         
 0   자전거번호     object        
 1   대여일시      datetime64[ns]
 2   대여 대여소번호  int64         
 3   대여 대여소명   object        
 4   이용시간      int64         
 5   이용거리      int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 101.4+ MB


In [4]:
bikes.describe()

,대여일시,대여 대여소번호,이용시간,이용거리
count,2215632,2.215632e+06,2.215632e+06,2.215632e+06
mean,2019-06-16 08:40:01.500303872,1.226154e+03,2.716998e+01,3.912916e+03
min,2019-06-01 00:00:00,3.000000e+00,1.000000e+00,0.000000e+00
25%,2019-06-08 22:04:00,5.050000e+02,8.000000e+00,1.220000e+03
50%,2019-06-16 15:13:00,1.159000e+03,1.700000e+01,2.270000e+03
75%,2019-06-23 20:22:00,1.846000e+03,3.700000e+01,4.670000e+03
max,2019-06-30 23:59:00,9.998000e+03,3.133000e+03,2.323100e+05
std,NaN,8.499197e+02,2.908119e+01,5.145471e+03


In [5]:
bike_seires = bikes['대여일시']
bike_df=bikes[['대여 대여소번호', '대여 대여소명']]
print(bike_seires.head())
print(bike_df.head())

0   2019-06-03 08:49:00
1   2019-06-03 08:33:00
2   2019-06-05 08:27:00
3   2019-06-05 08:46:00
4   2019-06-10 08:27:00
Name: 대여일시, dtype: datetime64[ns]
   대여 대여소번호            대여 대여소명
0       646  장한평역 1번출구 (국민은행앞)
1       526           용답토속공원 앞
2       646  장한평역 1번출구 (국민은행앞)
3       646  장한평역 1번출구 (국민은행앞)
4       529       장한평역 8번 출구 앞


In [6]:
요일 = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
bikes['대여일시'].dt.day_of_week.apply(lambda x: 요일[x])

0          월
1          월
2          수
3          수
4          월
          ..
2215627    월
2215628    화
2215629    수
2215630    목
2215631    금
Name: 대여일시, Length: 2215632, dtype: object

In [7]:
bikes['이용시간'].apply(lambda x: x/60)

0          0.450000
1          0.900000
2          0.200000
3          0.100000
4          0.083333
             ...   
2215627    0.216667
2215628    0.100000
2215629    0.116667
2215630    0.183333
2215631    0.100000
Name: 이용시간, Length: 2215632, dtype: float64

In [8]:
bike_shop = pd.read_excel('data/bike_rent_location.xlsx')
bike_shop.head()

,구분,대여소번호,대여소명,위도,경도,거치대수
0,마포구,101,101. (구)합정동 주민센터,37.549561,126.905754,5
1,마포구,102,102. 망원역 1번출구 앞,37.556000,126.910454,20
2,마포구,103,103. 망원역 2번출구 앞,37.554951,126.910835,14
3,마포구,104,104. 합정역 1번출구 앞,37.550629,126.914986,13
4,마포구,105,105. 합정역 5번출구 앞,37.550007,126.914825,5


In [9]:
bike_gu = bike_shop[['구분', '대여소번호', '대여소명', '위도', '경도']]

In [10]:
bikes = pd.merge(bikes, bike_gu, how='left', left_on='대여 대여소번호', right_on='대여소번호')
bikes.head(2)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,이용시간,이용거리,구분,대여소번호,대여소명,위도,경도
0,SPB-22040,2019-06-03 08:49:00,646,장한평역 1번출구 (국민은행앞),27,1330,동대문구,646.0,646. 장한평역 1번출구 (국민은행앞),37.561867,127.064377
1,SPB-07446,2019-06-03 08:33:00,526,용답토속공원 앞,54,1180,성동구,526.0,526. 용답토속공원 앞,37.563511,127.056725


In [11]:
bikes.drop(['대여소번호','대여소명'], axis='columns', inplace=True)
bikes.head(3)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,이용시간,이용거리,구분,위도,경도
0,SPB-22040,2019-06-03 08:49:00,646,장한평역 1번출구 (국민은행앞),27,1330,동대문구,37.561867,127.064377
1,SPB-07446,2019-06-03 08:33:00,526,용답토속공원 앞,54,1180,성동구,37.563511,127.056725
2,SPB-20387,2019-06-05 08:27:00,646,장한평역 1번출구 (국민은행앞),12,1930,동대문구,37.561867,127.064377


In [12]:
bikes.rename(columns={'구분':'대여구', '위도':'대여점위도', '경도':'대여점경도'}, inplace=True)
bikes.head(3)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,이용시간,이용거리,대여구,대여점위도,대여점경도
0,SPB-22040,2019-06-03 08:49:00,646,장한평역 1번출구 (국민은행앞),27,1330,동대문구,37.561867,127.064377
1,SPB-07446,2019-06-03 08:33:00,526,용답토속공원 앞,54,1180,성동구,37.563511,127.056725
2,SPB-20387,2019-06-05 08:27:00,646,장한평역 1번출구 (국민은행앞),12,1930,동대문구,37.561867,127.064377


ValueError: Could not interpret value `요일` for `x`. An entry with this name does not appear in `data`.